## Deliverable 2. Create a Customer Travel Destinations Map.

In [35]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from google_config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Resources/WeatherPy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Samarinda,-0.5000,117.1500,88.59,63,96,1.41,ID,2022-03-12 01:40:41
1,Hamilton,39.1834,-84.5333,30.36,92,100,11.50,US,2022-03-12 01:42:17
2,Tasiilaq,65.6145,-37.6368,33.96,93,100,30.31,GL,2022-03-12 01:45:35
3,Jamestown,42.0970,-79.2353,31.51,70,100,5.75,US,2022-03-12 01:44:09
4,Porto Walter,-8.2686,-72.7439,73.58,93,92,2.15,BR,2022-03-12 01:45:35


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Porto Walter,-8.2686,-72.7439,73.58,93,92,2.15,BR,2022-03-12 01:45:35
5,Saldanha,-33.0117,17.9442,68.00,81,0,3.78,ZA,2022-03-12 01:45:36
6,Cape Town,-33.9258,18.4232,73.11,88,0,2.30,ZA,2022-03-12 01:42:01
8,Castro,-24.7911,-50.0119,64.45,97,100,3.42,BR,2022-03-12 01:45:36
15,Hermanus,-34.4187,19.2345,69.84,81,0,5.97,ZA,2022-03-12 01:45:38
...,...,...,...,...,...,...,...,...,...
555,Beihai,21.4833,109.1000,69.66,83,100,10.04,CN,2022-03-12 01:48:23
561,Dunedin,-45.8742,170.5036,64.80,53,53,15.01,NZ,2022-03-12 01:48:25
566,Ambatofinandrahana,-20.5500,46.8000,61.07,95,100,4.27,MG,2022-03-12 01:48:26
567,Imbituba,-28.2400,-48.6703,72.82,95,100,14.99,BR,2022-03-12 01:48:26


In [25]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

Null values present in the Preferred cities list


In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# Populate None as the value under "Hotel Name" column initially
hotel_df["Hotel Name"] = None
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Porto Walter,BR,73.58,-8.2686,-72.7439,None
5,Saldanha,ZA,68.00,-33.0117,17.9442,None
6,Cape Town,ZA,73.11,-33.9258,18.4232,None
8,Castro,BR,64.45,-24.7911,-50.0119,None
15,Hermanus,ZA,69.84,-34.4187,19.2345,None
...,...,...,...,...,...,...
555,Beihai,CN,69.66,21.4833,109.1000,None
561,Dunedin,NZ,64.80,-45.8742,170.5036,None
566,Ambatofinandrahana,MG,61.07,-20.5500,46.8000,None
567,Imbituba,BR,72.82,-28.2400,-48.6703,None


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")

Hotel not found... skipping.17
Hotel not found... skipping.82
Hotel not found... skipping.147
Hotel not found... skipping.166
Hotel not found... skipping.179
Hotel not found... skipping.210
Hotel not found... skipping.217
Hotel not found... skipping.239
Hotel not found... skipping.283
Hotel not found... skipping.297
Hotel not found... skipping.306
Hotel not found... skipping.351
Hotel not found... skipping.408
Hotel not found... skipping.446
Hotel not found... skipping.471


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

#hotel_clean_df.to_csv(r'C:\Users\Owner\OneDrive\RUTGERS BOOTCAMP\M5 - API\Weather Challenge\WeatherPy_Vacation.csv', index = False)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Porto Walter,BR,73.58,-8.2686,-72.7439,Secretaria de saúde
5,Saldanha,ZA,68.00,-33.0117,17.9442,Blue Bay Lodge
6,Cape Town,ZA,73.11,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
8,Castro,BR,64.45,-24.7911,-50.0119,CHACARA BAILLY
15,Hermanus,ZA,69.84,-34.4187,19.2345,Misty Waves Boutique Hotel
...,...,...,...,...,...,...
555,Beihai,CN,69.66,21.4833,109.1000,21 Degree Hostel
561,Dunedin,NZ,64.80,-45.8742,170.5036,Scenic Hotel Southern Cross
566,Ambatofinandrahana,MG,61.07,-20.5500,46.8000,commune ambatofinandrahana
567,Imbituba,BR,72.82,-28.2400,-48.6703,Praia Hotel Imbituba


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))